In [2]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
# General imports
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from typing import NamedTuple, List, Tuple, Set, Union, Dict, Iterable
from pathlib import Path
from pprint import pprint

from crypto.algo.transpositions import DoubleTranspositionCipher
from crypto.algo.vigenere import VigenereCipher
from crypto.algo.caesar import CaesarCipher
from crypto.analysis.vigenere import VigenereAnalyzer
from crypto.analysis.caesar import CaesarAnalyzer
from crypto.analysis.hill import HillClimbingAnalyzer
from crypto.analysis.language.frequency import Language, ngram_fitness, GramStat, coincidence_index
from crypto.analysis.language.utils import abs_loss, log_loss
from crypto.utils import to_df
from crypto.analysis.transposition import TranspositionKeyGenerator
from pathlib import Path
from dataclasses import dataclass
from collections import defaultdict, OrderedDict
from IPython.display import display
np.set_printoptions(precision=3)
logging.basicConfig(format='[%(asctime)s] [%(name)s :%(levelname)s] - %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{
  "shell_port": 51401,
  "iopub_port": 51402,
  "stdin_port": 51403,
  "control_port": 51404,
  "hb_port": 51405,
  "ip": "127.0.0.1",
  "key": "cae83658-1fea45c337570062308d85db",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-19b590eb-9613-4370-a15e-94d351313e95.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned ON


In [3]:
#cipher = DoubleTranspositionCipher()
russian = Language(name="russian", root=Path(r"E:\Source\Personal\University\2019\Crypto\data"),
                   mapping={"Ё": "Е"}, encoding="utf-8")
english = Language(name="english", root=Path(r"E:\Source\Personal\University\2019\Crypto\data"), encoding="utf-8")

In [260]:
# 1.6.12
crypto = "TNOSSKAIMAGAEITMHETHTSRH--IHEU-D-\
NUEIDSATDTDDSARAHHENTTTDSOUIOEART\
FHDAOMWYWFERTNEONFDYAHSEIMEDGRWTA\
TISURUARTHJ"
crypt_size = (10, 11)
#
crypto_zachet = "ЯНЛВКРАДОЕТЕРГОМИЗЯЕ\
ЙЛТАЛФЫИПЕУИООГЕДБОР\
ЧРДЧИЕСМОНДКХИНТИКЕО\
НУЛАЕРЕБЫЫЕЕЗИОННЫЧД\
ЫТДОЕМППТЩВАНИПТЯЗСЛ\
ИКСИ-ТЧНО--Е-ЛУЛ-Т-Ж\
"
crypto_zachet_size = (6, 20)

crypto = crypto_zachet
crypt_size = crypto_zachet_size
    
df = to_df(crypto, crypt_size)

In [264]:
display(df)
ngram_fitness(cipher.to_text(df), language=russian, loss=log_loss, n=4)[1]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,Я,Н,Л,В,К,Р,А,Д,О,Е,Т,Е,Р,Г,О,М,И,З,Я,Е
1,Й,Л,Т,А,Л,Ф,Ы,И,П,Е,У,И,О,О,Г,Е,Д,Б,О,Р
2,Ч,Р,Д,Ч,И,Е,С,М,О,Н,Д,К,Х,И,Н,Т,И,К,Е,О
3,Н,У,Л,А,Е,Р,Е,Б,Ы,Ы,Е,Е,З,И,О,Н,Н,Ы,Ч,Д
4,Ы,Т,Д,О,Е,М,П,П,Т,Щ,В,А,Н,И,П,Т,Я,З,С,Л
5,И,К,С,И,-,Т,Ч,Н,О,-,-,Е,-,Л,У,Л,-,Т,-,Ж


1239.7329997997601

In [268]:
keygen = KeyGenerator([8, 2, 5, 6, 15, 3, 1, 11, 7, 0, 17, 19, 13, 14, 18, 10, 16, 9, 4, 12],
                      permutation_indices=range(0, 14), linked_groups=[[-1, -2, -3, -4, -5, -6]])
next_key = next(keygen)
analyzer = HillClimbingAnalyzer(cipher=cipher, key_generator=keygen,
                                loss_fn=log_loss, key_argname="k2", language=russian)
best_keys, best_loss = analyzer.fit(df, n=4)

04:23:45 INFO:[GREAT SUCCESS] Best key is updated: [19, 3, 5, 11, 15, 8, 13, 14, 7, 0, 17, 6, 2, 1, 18, 10, 16, 9, 4, 12] (930.104)
04:23:47 INFO:[GREAT SUCCESS] Best key is updated: [0, 14, 19, 3, 5, 11, 15, 8, 7, 17, 6, 2, 13, 1, 18, 10, 16, 9, 4, 12] (919.722)
04:23:48 INFO:Phase [5/100 5.00%]...
04:23:48 INFO:Decoded so far: 'ЯОЕВРЕМОДЗАЛГНЯТИЕКРЙГРАФИЕПИБЫТОЛОУДЕЛОЧНОЧЕКТОМКСДИРЕДИНИХНОДАРЕНЫБЫЕЛИУЧЕНЫЕЗЫПЛОМАТТПЗПДИТСВЯЩЕНИУЖИТЕЛОНТЧСЛК------' (919.722)
04:23:50 INFO:[GREAT SUCCESS] Best key is updated: [2, 6, 0, 17, 11, 15, 8, 7, 19, 3, 5, 14, 13, 1, 18, 10, 16, 9, 4, 12] (901.310)
04:23:54 INFO:Phase [10/100 10.00%]...
04:23:54 INFO:Decoded so far: 'ЛАЯЗЕМОДЕВРОГНЯТИЕКРТЫЙБИЕПИРАФГОЛОУДЕЛОДСЧККТОМОЧЕНИРЕДИНИХЛЕНЫЕНЫБДАРОИУЧЕНЫЕЗДПЫЗАТТПЛОМПИТСВЯЩЕНСЧИТЕЛОНЖИТУЛК------' (901.310)
04:23:59 INFO:[GREAT SUCCESS] Best key is updated: [7, 8, 2, 11, 15, 0, 17, 13, 14, 19, 3, 5, 6, 1, 18, 10, 16, 9, 4, 12] (890.244)
04:24:01 INFO:[GREAT SUCCESS] Best key is updated: [13, 14, 19, 3, 5, 

KeyboardInterrupt: 

In [336]:
## Задача 92 - Зачёт
text = "12 2 24 5 3 21  6 29  28 2 20 18  20 21 5 10  27 17 2 11 2 16  -  19 2  27 5  8 29 12 31 22 2 16  ,  \
19 2  19 5  17 29 8 29 6 29 16  8 2 19 19 29  10  19 29 14 19 29  29 19 10  2 24 2 11 2 16  10 14 18 21  \
17 2 20 2 28 29 16  \
21 29 28 6 29 16 ."
text = text.split("  ")
words = []
for word in text:
    word_code = []
    for character in word.split(" "):
        try:
            word_code.append(chr(ord('А') + int(character)))
        except ValueError:
            word_code.append(character)
    if word_code:
        words.append(''.join(word_code))
crypttext = ' '.join(words)
crypttext

'МВШЕГХ ЖЭ ЬВФТ ФХЕК ЫСВЛВР - УВ ЫЕ ИЭМЯЦВР , УВ УЕ СЭИЭЖЭР ИВУУЭ К УЭОУЭ ЭУК ВШВЛВР КОТХ СВФВЬЭР ХЭЬЖЭР.'

In [337]:
russian[1].ngrams

OrderedDict([('О', 10.607471474100134),
             ('Е', 8.431924637454106),
             ('А', 8.036776086839842),
             ('И', 7.977309760439354),
             ('Н', 6.719308679558824),
             ('Т', 5.828381589673569),
             ('С', 5.70694219229602),
             ('Р', 5.38024551515306),
             ('В', 4.745375997274494),
             ('Л', 4.321998028778656),
             ('К', 3.4860271596901606),
             ('М', 3.108986099059515),
             ('Д', 2.953990310281136),
             ('П', 2.822935006043711),
             ('У', 2.282580270978222),
             ('Я', 1.997498643435968),
             ('Ы', 1.9111193827779187),
             ('Г', 1.8766858380065394),
             ('З', 1.6080116506163236),
             ('Б', 1.549070938026797),
             ('Ь', 1.3852738495995076),
             ('Й', 1.3597278880345371),
             ('Ч', 1.2322595683708366),
             ('Х', 1.0204066830420802),
             ('Ж', 0.8043752621122098),
             ('Ю'

In [345]:
def decrypt(text: str, mapping: Dict[str, str]=None) -> Tuple[str, str]:
    if not mapping:
        return text
    reversed_text = ''.join([mapping.get(char, char) for char in text])
    reversed_pos = ''.join([char if not char.isalpha() else ('_' if char not in mapping else mapping[char]) 
                            for char in text])
    return reversed_text, reversed_pos

def make_keytable(conversions: Iterable[Tuple[str, str]], add_reversions: bool=True) -> Dict[str, str]:
    table = {}
    reverse_table = {}
    for conversion in conversions:
        for ch_from, ch_to in zip(*conversion):
            if ch_from in table and table[ch_from] != ch_to:
                raise ValueError(f"Conversions are invalid: {conversion} has character mapping "
                                 f"{ch_from} -> {ch_to}, "
                                 f"but another conversion set {ch_from} -> {table[ch_from]}")
            table[ch_from] = ch_to
            reverse_table[ch_to] = ch_from
    if add_reversions:
        for ch_to, ch_from in reverse_table.items():
            if ch_to not in table:
                table[ch_to] = ch_from
    return table
        

def evaluate_key(crypttext, key=None, ngram=1):
    data, positions = decrypt(crypttext, key)
    logger.debug(f"Decrypted: {data}")
    logger.debug(f"Decrypted: {positions}")
    logger.debug(f"Decrypted stats: ")
    stat = GramStat.from_text("".join([val if positions[idx] == '_' else '_' for idx, val in enumerate(data)]), 
                              n=ngram, sort=True, percentage=True)
    print(stat.ngrams)
    return log_loss(stat, russian[ngram])

class SubstitutionKeyGenerator(KeyGenerator):
    
    def __init__()
    
    def __next__()
    
    
    def hill_climbing()
    
    

In [362]:
kt = make_keytable([("ЭУК", "ОНИ"), ("УВ", "НЕ"), ("Е", "А"), ("Ы", "З"), ("О", "Щ"), ("И", "Д"), ("ТХ", "УТ"), ("Ф", "С"), 
                    ("Р", "Й"), ("ИЭМЯЦВР", "ДОБЫЧЕЙ"), ("ЬВФТ", "ЛЕСУ"), ("С", "В"), ("Л", "Р"), ("Ж", "П"), ("Ш", "Г"), ("Г", "Ю")], False)
print("Keytable values")
print(list(kt.values()))
evaluate_key(crypttext,  kt, 1)

Keytable values
['О', 'Н', 'И', 'Е', 'А', 'З', 'Щ', 'Д', 'У', 'Т', 'С', 'Й', 'Б', 'Ы', 'Ч', 'Л', 'В', 'Р', 'П', 'Г', 'Ю']
Decrypted: БЕГАЮТ ПО ЛЕСУ СТАИ ЗВЕРЕЙ - НЕ ЗА ДОБЫЧЕЙ , НЕ НА ВОДОПОЙ ДЕННО И НОЩНО ОНИ ЕГЕРЕЙ ИЩУТ ВЕСЕЛОЙ ТОЛПОЙ.
Decrypted: БЕГАЮТ ПО ЛЕСУ СТАИ ЗВЕРЕЙ - НЕ ЗА ДОБЫЧЕЙ , НЕ НА ВОДОПОЙ ДЕННО И НОЩНО ОНИ ЕГЕРЕЙ ИЩУТ ВЕСЕЛОЙ ТОЛПОЙ.
Decrypted stats: 
OrderedDict([('_', 100.0)])


3.3945840734602695

In [9]:
import collections
import datetime

crypttext = "ТПИЁРЖЕМААСФСГЬОГХЖПН"
alphabet = list(sorted(russian[1].keys()))


best_keys_alltime = collections.deque(maxlen=5)

def decrypt(crypttext, key: str):
    start = ord(alphabet[0])
    plaintext = ""
    for idx, c in enumerate(crypttext):
        key_char = key[idx % len(key)]
        plaintext_char = (ord(c) - start - int(key_char))
        plaintext += alphabet[plaintext_char]
    #logger.info(f"Decoded plaintext: {plaintext}")
    return plaintext


def brute_force(crypttext: str, start: datetime.datetime, stop=datetime.datetime(year=2099, month=12, day=31), n=3):
    date = start
    best_keys, best_loss = [], None
    aday = datetime.timedelta(days=1)
    while date < stop:
        key = date.strftime("%d%m%y")
        logger.debug(f"Trying key: {key}...")
        text = decrypt(crypttext, key)
        stat = GramStat.from_text(text, n=n)
        loss = log_loss(stat, russian[n])
        if not best_keys or loss < best_loss:
            logger.info(f"[GREAT SUCCESS!] {key} is the best key ({loss:.3f})...")
            logger.info(f"Text is '{text}'")
            best_keys_alltime.append((best_keys, best_loss))
            best_keys, best_loss = [key], loss
        elif np.equal(best_loss, loss):
            logger.info(f"[SUCCESS!] Adding {key} to best keys ({loss:.3f})...")
            best_keys.append(key)
        date += aday
    return best_keys, best_loss
            
brute_force(crypttext, start=datetime.datetime(year=2000, month=1, day=1), n=4)
  

05:52:02 INFO:[GREAT SUCCESS!] 010100 is the best key (216.484)...
05:52:02 INFO:Text is 'ТОИРРЖЕЛАЯСФСВЬНГХЖОН'
05:52:02 INFO:[GREAT SUCCESS!] 020100 is the best key (214.693)...
05:52:02 INFO:Text is 'ТНИРРЖЕКАЯСФСБЬНГХЖНН'
05:52:02 INFO:[GREAT SUCCESS!] 120100 is the best key (210.129)...
05:52:02 INFO:Text is 'СНИРРЖДКАЯСФРБЬНГХЕНН'
05:52:02 INFO:[GREAT SUCCESS!] 010900 is the best key (209.625)...
05:52:02 INFO:Text is 'ТОИИРЖЕЛАЧСФСВЬЕГХЖОН'
05:52:02 INFO:[GREAT SUCCESS!] 020900 is the best key (204.268)...
05:52:02 INFO:Text is 'ТНИИРЖЕКАЧСФСБЬЕГХЖНН'
05:52:02 INFO:[GREAT SUCCESS!] 010101 is the best key (201.411)...
05:52:02 INFO:Text is 'ТОИРРЕЕЛАЯСУСВЬНГФЖОН'
05:52:02 INFO:[GREAT SUCCESS!] 020101 is the best key (197.220)...
05:52:02 INFO:Text is 'ТНИРРЕЕКАЯСУСБЬНГФЖНН'
05:52:02 INFO:[GREAT SUCCESS!] 100101 is the best key (186.276)...
05:52:02 INFO:Text is 'СПИРРЕДМАЯСУРГЬНГФЕПН'
05:52:02 INFO:[GREAT SUCCESS!] 110101 is the best key (184.908)...
05:52:02 INFO:Text is 'СОИРРЕ

(['311207'], 134.9852925160941)

In [370]:
-1 % 3

2

# 2nd HW task (common)

In [21]:
sub_text = "EMGLOSUDCGDNCUSWYSFHNSFCYKDPUMLWGYICOXYSIPJCK"\
"QPKUGKMGOLICGINCGACKSNISACYKZSCKXECJCKSHYSXCG"\
"OIDPKZCNKSHICGIWYGKKGKGOLDSILKGOIUSIGLEDSPWZU"\
"GFZCCNDGYYSFUSZCNXEOJNCGYEOWEUPXEZGACGNFGLKNS"\
"ACIGOIYCKXCJUCIUZCFZCCNDGYYSFEUEKUZCSOCFZCCNC"\
"IACZEJNCSHFZEJZEGMXCYHCJUMGKUCY"

## ENGLISH NGRAMS VIEW

In [102]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

pprint(english[1].ngrams)
pprint(OrderedDict(islice(english[2].ngrams.items(), 100)))
pprint(OrderedDict(islice(english[3].ngrams.items(), 100)))
pprint(OrderedDict(islice(english[4].ngrams.items(), 1000)))

OrderedDict([('E', 12.09652247516903),
             ('T', 8.938126949659495),
             ('A', 8.551690673195276),
             ('O', 7.467265410810447),
             ('I', 7.32511860723129),
             ('N', 7.172184876283856),
             ('S', 6.728203117491646),
             ('R', 6.332710132840231),
             ('H', 4.955707280570642),
             ('L', 4.206464329306453),
             ('D', 3.8711837357374175),
             ('C', 3.164435380900101),
             ('U', 2.681580936230437),
             ('M', 2.5263217360184447),
             ('F', 2.1815103969122527),
             ('G', 2.0863354250923156),
             ('P', 2.0661660788966265),
             ('W', 1.82536189504165),
             ('Y', 1.7213606152473404),
             ('B', 1.6047959168228292),
             ('V', 1.059346274662571),
             ('K', 0.8086975227142329),
             ('J', 0.2197788956104563),
             ('X', 0.1913504859413457),
             ('Z', 0.11375632147038378),
             ('

In [124]:
def decrypt_sub(text: str, mapping: Dict[str, str]=None) -> Tuple[str, str]:
    if not mapping:
        return text
    reversed_text = ''.join([mapping.get(char, char) for char in text])
    reversed_pos = ''.join([char if not char.isalpha() else ('_' if char not in mapping else mapping[char]) 
                            for char in text])
    return reversed_text, reversed_pos

def make_sub_keytable(conversions: Iterable[Tuple[str, str]], add_reversions: bool=True) -> Dict[str, str]:
    table = {}
    reverse_table = {}
    for conversion in conversions:
        for ch_from, ch_to in zip(*conversion):
            if ch_from in table and table[ch_from] != ch_to:
                raise ValueError(f"Conversions are invalid: {conversion} has character mapping "
                                 f"{ch_from} -> {ch_to}, "
                                 f"but another conversion set {ch_from} -> {table[ch_from]}")
            table[ch_from] = ch_to
            reverse_table[ch_to] = ch_from
    if add_reversions:
        for ch_to, ch_from in reverse_table.items():
            if ch_to not in table:
                table[ch_to] = ch_from
    return table
        

def evaluate_sub_key(crypttext, language: Language, key=None, ngram=1):
    data, positions = decrypt_sub(crypttext, key)
    logger.info(f"Encrypted: {crypttext}")
    logger.info(f"Decrypted: {data}")
    logger.info(f"Decrypted: {positions}")
    logger.info(f"Decrypted stats: ")
    #stat = GramStat.from_text("".join([val if positions[idx] == '_' else '_' for idx, val in enumerate(data)]), 
    #                          n=ngram, sort=True, percentage=True)
    stat = GramStat.from_text(data, n=ngram, sort=True, percentage=language.percentage)
    return log_loss(stat, language[ngram])

In [168]:
kt = make_sub_keytable([
    #("C", "C"),
    ("C", "E"),
    ("UZC", "THE"),
    ("NC", "LE"),
    ("F", "W"),
    ("S", "O"),
    ("O", "N"),
    ("Y", "R"),
    ("G", "A"),
    ("D", "B"),
    ("W", "G"),
    ("H", "F"),
    ("I", "D"),
    ("A", "V"),
    ("P", "U"),
    ("X", "P"),
    ("L", "Y"),
    ("E", "I"),
    ("M", "M"),
    ("J", "C"),
    ("K", "S"),
    ("Q", "J")
], False)

# S -> O, S
# NC -> RE TE SE LE
#for from_what, to_what in sub_text_subs:
 #   working_sub_text = working_sub_text.replace(from_what, to_what)

# NO DECRYPTION: log loss = 1839.907
evaluate_sub_key(sub_text, english, kt, 3)

[08:16:20] [__main__ :INFO] - Encrypted: EMGLOSUDCGDNCUSWYSFHNSFCYKDPUMLWGYICOXYSIPJCKQPKUGKMGOLICGINCGACKSNISACYKZSCKXECJCKSHYSXCGOIDPKZCNKSHICGIWYGKKGKGOLDSILKGOIUSIGLEDSPWZUGFZCCNDGYYSFUSZCNXEOJNCGYEOWEUPXEZGACGNFGLKNSACIGOIYCKXCJUCIUZCFZCCNDGYYSFEUEKUZCSOCFZCCNCIACZEJNCSHFZEJZEGMXCYHCJUMGKUCY
[08:16:20] [__main__ :INFO] - Decrypted: IMAYNOTBEABLETOGROWFLOWERSBUTMYGARDENPRODUCESJUSTASMANYDEADLEAVESOLDOVERSHOESPIECESOFROPEANDBUSHELSOFDEADGRASSASANYBODYSANDTODAYIBOUGHTAWHEELBARROWTOHELPINCLEARINGITUPIHAVEALWAYSLOVEDANDRESPECTEDTHEWHEELBARROWITISTHEONEWHEELEDVEHICLEOFWHICHIAMPERFECTMASTER
[08:16:20] [__main__ :INFO] - Decrypted: IMAYNOTBEABLETOGROWFLOWERSBUTMYGARDENPRODUCESJUSTASMANYDEADLEAVESOLDOVERSHOESPIECESOFROPEANDBUSHELSOFDEADGRASSASANYBODYSANDTODAYIBOUGHTAWHEELBARROWTOHELPINCLEARINGITUPIHAVEALWAYSLOVEDANDRESPECTEDTHEWHEELBARROWITISTHEONEWHEELEDVEHICLEOFWHICHIAMPERFECTMASTER
[08:16:20] [__main__ :INFO] - Decrypted stats: 


768.9437527639137

In [98]:
pprint(sub_text_grams[1].ngrams)
pprint(sub_text_grams[2].ngrams)
pprint(sub_text_grams[3].ngrams)
pprint(sub_text_grams[4].ngrams)

OrderedDict([('C', 14.453125),
             ('G', 9.375),
             ('S', 7.8125),
             ('K', 7.03125),
             ('Y', 5.859375),
             ('I', 5.859375),
             ('U', 5.46875),
             ('N', 5.078125),
             ('Z', 5.078125),
             ('E', 4.6875),
             ('O', 3.90625),
             ('F', 3.515625),
             ('D', 3.125),
             ('L', 2.734375),
             ('X', 2.734375),
             ('J', 2.734375),
             ('P', 2.34375),
             ('M', 1.953125),
             ('W', 1.953125),
             ('H', 1.953125),
             ('A', 1.953125),
             ('Q', 0.390625)])
OrderedDict([('CG', 2.7450980392156863),
             ('ZC', 2.7450980392156863),
             ('NC', 1.9607843137254901),
             ('YS', 1.9607843137254901),
             ('CK', 1.9607843137254901),
             ('GO', 1.9607843137254901),
             ('AC', 1.9607843137254901),
             ('CN', 1.9607843137254901),
             ('SF', 1.56

In [12]:
cipher = CaesarCipher(english.alphabet)
cipher.decrypt(text, list(result.best_keys)[0])

'GOINQUWFEIFPEWUYAUHJPUHEAMFRWONYIAKEQZAUKRLEMSRMWIMOIQNKEIKPEICEMUPKUCEAMBUEMZGELEMUJAUZEIQKFRMBEPMUJKEIKYAIMMIMIQNFUKNMIQKWUKINGFURYBWIHBEEPFIAAUHWUBEPZGQLPEIAGQYGWRZGBICEIPHINMPUCEKIQKAEMZELWEKWBEHBEEPFIAAUHGWGMWBEUQEHBEEPEKCEBGLPEUJHBGLBGIOZEAJELWOIMWEA'

# 3rd HW task (common)

In [23]:
vigenere_analyzer = VigenereAnalyzer(english)

In [24]:
vigenere_text = "KCCPKBGUFDPHQTYAVINRRTMVGRKDNBVFDETDGILTXRGUD"\
"DKOTFMBPVGEGLTGCKQRACQCWDNAWCRXIZAKFTLEWRPTYC"\
"QKYVXCHKFTPONCQQRHJVAJUWETMCMSPKQDYHJVDAHCTRL"\
"SVSKCGCZQQDZXGSFRLSWCWSJTBHAFSIASPRJAHKJRJUMV"\
"GKMITZHFPDISPZLVLGWTFPLKKEBDPGCEBSHCTJRWXBAFS"\
"PEZQNRWXCVYCGAONWDDKACKAWBBIKFTIOVKCGGHJVLNHI"\
"FFSQESVYCLACNVRWBBIREPBBVFEXOSCDYGZWPFDTKFQIY"\
"CWHJVLNHIQIBTKHJVNPIST"

vigenere_result = vigenere_analyzer.fit(vigenere_text)

06:09:16 INFO:Trying out length 6 with index 0.07951007426670266...
06:09:16 INFO:Key E is the best key so far (score = 0.034 vs. best_score = None)
06:09:16 INFO:Key O is the best key so far (score = 0.038 vs. best_score = 0.03449529061727547)
06:09:16 INFO:Key I is the best key so far (score = 0.043 vs. best_score = 0.038155890964184536)
06:09:16 INFO:Key C is the best key so far (score = 0.063 vs. best_score = 0.042505382384423016)
06:09:16 INFO:Key E is the best key so far (score = 0.041 vs. best_score = None)
06:09:16 INFO:Key N is the best key so far (score = 0.048 vs. best_score = 0.0405703382851644)
06:09:16 INFO:Key R is the best key so far (score = 0.070 vs. best_score = 0.048013166522720085)
06:09:16 INFO:Key E is the best key so far (score = 0.035 vs. best_score = None)
06:09:16 INFO:Key A is the best key so far (score = 0.036 vs. best_score = 0.03508285065048313)
06:09:16 INFO:Key O is the best key so far (score = 0.037 vs. best_score = 0.03567496317792044)
06:09:16 INFO:K

06:09:17 INFO:Key C is the best key so far (score = 0.049 vs. best_score = 0.047965245385907636)
06:09:17 INFO:Key E is the best key so far (score = 0.035 vs. best_score = None)
06:09:17 INFO:Key T is the best key so far (score = 0.038 vs. best_score = 0.03461634448895995)
06:09:17 INFO:Key O is the best key so far (score = 0.048 vs. best_score = 0.03833629477865236)
06:09:17 INFO:Key G is the best key so far (score = 0.048 vs. best_score = 0.047624545051310516)
06:09:17 INFO:Key E is the best key so far (score = 0.033 vs. best_score = None)
06:09:17 INFO:Key T is the best key so far (score = 0.036 vs. best_score = 0.033411825236582995)
06:09:17 INFO:Key A is the best key so far (score = 0.038 vs. best_score = 0.036378032128125536)
06:09:17 INFO:Key I is the best key so far (score = 0.041 vs. best_score = 0.03805528762262012)
06:09:17 INFO:Key S is the best key so far (score = 0.043 vs. best_score = 0.040741491838879325)
06:09:17 INFO:Key C is the best key so far (score = 0.043 vs. bes

06:09:17 INFO:Key R is the best key so far (score = 0.045 vs. best_score = 0.0429883929155447)
06:09:17 INFO:Key C is the best key so far (score = 0.045 vs. best_score = 0.04470848651156255)
06:09:17 WARNING:Caesar analyzer for cut index 6 @ length 7 returned 2 keys ({'C', 'Z'}). We'll select only the first one (C)
06:09:17 INFO:Trying out length 3 with index 0.05921910403091459...
06:09:17 INFO:Key E is the best key so far (score = 0.036 vs. best_score = None)
06:09:17 INFO:Key T is the best key so far (score = 0.037 vs. best_score = 0.03574215298005951)
06:09:17 INFO:Key A is the best key so far (score = 0.039 vs. best_score = 0.036987072611727206)
06:09:17 INFO:Key I is the best key so far (score = 0.041 vs. best_score = 0.03854067704524419)
06:09:17 INFO:Key S is the best key so far (score = 0.041 vs. best_score = 0.04058380651167954)
06:09:17 INFO:Key L is the best key so far (score = 0.043 vs. best_score = 0.040779189341236054)
06:09:17 INFO:Key C is the best key so far (score = 

In [27]:
vigenere_cipher = VigenereCipher(english.alphabet)
vigenere_cipher.decrypt(vigenere_text, list(vigenere_result.best_keys)[0])

'ILEARNEDHOWTOCALCULATETHEAMOUNTOFPAPERNEEDEDFORAROOMWHENIWASATSCHOOLYOUMULTIPLYTHESQUAREFOOTAGEOFTHEWALLSBYTHECUBICCONTENTSOFTHEFLOORANDCEILINGCOMBINEDANDDOUBLEITYOUTHENALLOWHALFTHETOTALFOROPENINGSSUCHASWINDOWSANDDOORSTHENYOUALLOWTHEOTHERHALFFORMATCHINGTHEPATTERNTHENYOUDOUBLETHEWHOLETHINGAGAINTOGIVEAMARGINOFERRORANDTHENYOUORDERTHEPAPER'

# 4th HW task (common)

In [8]:
unknown_analyzer = VigenereAnalyzer(english)
unknown_text = "BNVSNSIHQCEELSSKKYERIFJKXUMBGYKAMQLJTYAVFBKVT\
DVBPVVRJYYLAOKYMPQSCGDLFSRLLPROYGESEBUUALRWXM\
MASAZLGLEDFJBZAVVPXWICGJXASCBYEHOSNMULKCEAHTQ\
OKMFLEBKFXLRRFDTZXCIWBJSICBGAWDVYDHAVFJXZIBKC\
GJIWEAHTTOEWTUHKRQVVRGZBXYIREMMASCSPBNLHJMBLR\
FFJELHWEYLWISTFVVYFJCMHYUYRUFSFMGESIGRLWALSWM\
NUHSIMYYITCCQPZSICEHBCCMZFEGVJYOCDEMMPGHVAAUM\
ELCMOEHVLTIPSUYILVGFLMVWDVYDBTHFRAYISYSGKVSUU\
HYHGGCKTMBLRX"
print(coincidence_index(unknown_text, language=english))

#unknown_result = unknown_analyzer.fit(unknown_text)

0.040575818539026365


In [5]:
unknown_cipher = VigenereCipher(english.alphabet)
unknown_cipher.decrypt(unknown_text, list(unknown_result.best_keys)[0])

'IGREWUPAMONGSLOWTALKERSMENINPARTICULARWHODROPPEDWORDSAFEWATATIMELIKEBEANSINAHILLANDWHENIGOTTOMINNEAPOLISWHEREPEOPLETOOKALAKEWOBEGONCOMMATOMEANTHEENDOFASTORYICOULDNTSPEAKAWHOLESENTENCEINCOMPANYANDWASCONSIDEREDNOTTOOBRIGHTSOIENROLLEDINASPEECHCOURSETAUGHTBYORVILLESANDTHEFOUNDEROFREFLEXIVERELAXOLOGYASELFHYPNOTICTECHNIQUETHATENABLEDAPERSONTOSPEAKUPTOTHREEHUNDREDWORDSPERMINUTE'

# Differential cryptoanalysis for dummies

# Hill climbing for TSEHVAIESSRYIYQ.

In [59]:
kg = TranspositionKeyGenerator([0, 3, 6, 1, 12, 9, 4, 7, 10, 13, 2, 8, 5, 11, 14],
                               linked_groups=[[0,1,2,3],[4,5,6,7,8,9]], shuffle_linked_groups_outer=False)
analyzer = HillClimbingAnalyzer(DoubleTranspositionCipher(), key_generator=kg, language=english, key_argname="k2")
encrypted = to_df("TSEHVAIESSRYIYQ", (1,15))
encrypted
analyzer.fit(encrypted, ngrams=4, phases=100)



[02:57:55] [crypto.analysis.hill :INFO] - [GREAT SUCCESS] Best key is updated: [0, 3, 6, 1, 12, 9, 4, 7, 10, 13, 2, 5, 8, 11, 14] (68.978)
[02:57:55] [crypto.analysis.hill :INFO] - Phase [5/100 5.00%]...
[02:57:55] [crypto.analysis.hill :INFO] - Decoded so far: 'THISISVERYEASYQ' (68.978)
[02:57:55] [crypto.analysis.hill :INFO] - Phase [10/100 10.00%]...
[02:57:55] [crypto.analysis.hill :INFO] - Decoded so far: 'THISISVERYEASYQ' (68.978)
[02:57:56] [crypto.analysis.hill :INFO] - Phase [15/100 15.00%]...
[02:57:56] [crypto.analysis.hill :INFO] - Decoded so far: 'THISISVERYEASYQ' (68.978)
[02:57:56] [crypto.analysis.hill :INFO] - Phase [20/100 20.00%]...
[02:57:56] [crypto.analysis.hill :INFO] - Decoded so far: 'THISISVERYEASYQ' (68.978)
[02:57:56] [crypto.analysis.hill :INFO] - Phase [25/100 25.00%]...
[02:57:56] [crypto.analysis.hill :INFO] - Decoded so far: 'THISISVERYEASYQ' (68.978)
[02:57:56] [crypto.analysis.hill :INFO] - Phase [30/100 30.00%]...
[02:57:56] [crypto.analysis.hill :IN

AnalyzerResult(best_keys=[[14, 11, 2, 5, 8, 0, 3, 6, 1, 12, 9, 4, 7, 10, 13], [14, 11, 2, 5, 8, 0, 3, 6, 1, 12, 9, 4, 7, 10, 13]], best_score=65.54070422226174, score_is_loss=True)